<img src="img/Asesoftware_logo.png" width="200" height="100">
## <center>MODELO WORD2VEC (WORD EMBEDDINGS) EN DOCUMENTOS POSTMORTEM</center>
<center>
    <h3>Autor: <i>Álvaro Valbuena</i></h3>
    <h4><i>avalbuena@asesoftware.com</i></h4>
    <h3>Área de Innovación</h3>
</center>

__Representaciones Vectoriales con Word2Vec__

_Word2Vec_ es una aproximación que nos ayuda a crear vectores similares para palabras similares. Las palabras que estan relacionadas son mapeadas a puntos que estan cercanos entre si en un espacio dimensional. Los modelos Word2Vec tiene las siguientes ventajas:

- Wor2Vec se construye sobre el hecho de que las palabras que comparten contextos también comparten significados semánticos.

- Los modelos Wor2Vec predicen una palabra usando sus vecinos a travez del aprendizaje de vectores densos llamados __embeddings__.

- Los modelos Wor2Vec tambien son eficientes computacionalmente.

- Wor2Vec son modelos no supervizados que aprenden de embeddings de texto bruto.

- Son dos los modelos Wor2Vec: __CBOW__ (Cuando la palabra target es predicha usando las palabras contexto) y __Skip-gramv__ (Cuando las palabras contexto son predichas usando la palabra target)

Word2vec es una red neuronal de 3 capas (Capa de entrada, capa oculta y capa de salida). La capa intermedia (capa oculta) contruye una representación latente para que las palabras de entrada se transformen en la representación del vector de salida.

En la representación vectorial de las palabras de Word2Vec se pueden encontrar relaciones matemáticas interesantes como

<center>$king - man = queen - woman$</center>

Ejemplo:

Tenemos las siguientes dos frases:

- I like watching a movie.
- I enjoy watching a movie.

Siguiendo el modelo CBOW y tomando las palabras contexto como entrada y tratamos de predecir la palabra target, entonces la salida sería como sigue:

![cbow_01](img/cbow_01.jpg)

Una forma vectorizada de la entrada y salida luciría algo así:

![cbow_02](img/cbow_02.png)

En la red neuronal para nuestro ejemplo , habrían 3 neuronas en la capa oculta y en la capa de salida habrían 5 neuronas con funciones _softmax_ de modo que nos darán las probabilidades de las palabras.

![NN](img/nn.png)

### <CENTER>CODIGO EN PYTHON DE WORD2VEC SOBRE LOS DOCUMENTOS POSTMORTEM

### Librerías necesarias

#### Descargar paquetes

In [ ]:
!conda install -c anaconda pandas -y

In [ ]:
!conda install -c anaconda numpy -y

In [ ]:
!conda install -c anaconda gensim -y

In [ ]:
!conda install -c conda-forge multiprocess -y

In [ ]:
!conda install -c conda-forge spacy -y

In [ ]:
!python -m spacy download es_core_news_md

In [1]:
import warnings
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np

from gensim.models import Word2Vec
import multiprocessing

import spacy

Nos ayudaremos de spacy para obtener una lista de stopwords

In [2]:
nlp = spacy.load('es_core_news_md', disable=['tagger', 'parser', 'ner'])

Algunas funciones necesarias

In [3]:
#Esta función concatena una serie de pandas
def concat_text(pdSeries):
    pdSeries = pdSeries.str.rstrip('.')
    return pdSeries.str.cat(sep='. ')


#Esta función lematiza y elimina las stopwords
def cleaning(doc):    
    txt = [token.text for token in doc if not token.is_stop]
    if len(txt) > 2:
        return ' '.join(txt)
    

#Esta función filtra stopwords de una lista
def filter_stopwords(answ, stopwords):
    ans_flt = [token for token in answ if not token[0] in stopwords]
    return ans_flt

Carga y visualización de los datos

In [4]:
data = pd.read_excel("data\REPOSITORIO_LECCIONES APRENDIDAS.xlsx", sheet_name=0)
data.head()

,PROYECTO,TIPO PROYECTO,CLIENTE,FECHA DE INICIO,FECHA FINAL,CONTEXTO,LECCIONES APRENDIDAS,Davi Cargue,SWEBOK,Practices,Paradigma,Tecnologia,Equipo trabajo,Aporte,IMPACTO,FASE,DURACION,Selección
0,CO_625 SODIMAC FABRICA,MANTENIMIENTO,SODIMAC,2018-02-15,2018-09-19,Se planeo dentro del sprint la participación d...,Se deben incluir revisiones de UX en los proce...,NaN,"Requirements, Quality",NaN,UX,NaN,NaN,NaN,POSITIVO,CALIDAD,513,NaN
1,CO_625 SODIMAC FABRICA,MANTENIMIENTO,SODIMAC,2018-02-15,2018-09-19,Integrantes de parte de Sodimac debian constru...,Asegurar que la planeación y ejecución del Spr...,NaN,"Design, Management",NaN,NaN,NaN,NaN,NaN,POSITIVO,CONSTRUCCION,513,NaN
2,CO_625 SODIMAC FABRICA,MANTENIMIENTO,SODIMAC,2018-02-15,2018-09-19,Se presentó el cambio de tres analistas de cal...,Se presentan demoras en el acoplamiento del eq...,NaN,Management,NaN,NaN,NaN,Rotación,NaN,NEGATIVO,CALIDAD,513,NaN
3,CO_625 SODIMAC FABRICA,MANTENIMIENTO,SODIMAC,2018-02-15,2018-09-19,Cuando en pruebas de aceptación se generaban d...,Se estableció la generación de ramas para cada...,NaN,ConfigurationManagement,NaN,NaN,NaN,NaN,NaN,POSITIVO,CONSTRUCCION,513,NaN
4,CO_625 SODIMAC FABRICA,MANTENIMIENTO,SODIMAC,2018-02-15,2018-09-19,"Asesoftware participó en la definición,\n cons...","La definición, modelo de implementación técnic...",NaN,NaN,NaN,Pasarela pagos,NaN,Dominio tecnologico,NaN,POSITIVO,CONSTRUCCION,513,NaN


Como podemos ver, no hay vacíos en las columnas "CONTEXTO" y "LECCIONES APRENDIDAS"

In [5]:
data[['CONTEXTO','LECCIONES APRENDIDAS']].isna().sum()

CONTEXTO                0
LECCIONES APRENDIDAS    0
dtype: int64

Ahora tomaremos solo el contexto y las lecciones aprendidas. Esto es debido a que nos interesan las lecciones aprendidas pero estas ocurren bajo un contexto

In [6]:
data = data[['CONTEXTO','LECCIONES APRENDIDAS']]

Ahora crearemos una nueva columna en el dataframe que se llamará __CONTEX_LECC__ la cual tendrá la concatenación del contexto con la lección aprendida. También eliminaremos los saltos de línea y reemplazaremos los multiples espacios por uno solo. Finalmente, habrá un punto seguido entre un contexto y una lección aprendida.

In [7]:
data["CONTEX_LECC"] = [concat_text(i[1]) for i in data[['CONTEXTO', 'LECCIONES APRENDIDAS']].iterrows()]
data.CONTEX_LECC = data.CONTEX_LECC.str.replace('\n', ' ').replace('\s+', ' ')
data.head()

,CONTEXTO,LECCIONES APRENDIDAS,CONTEX_LECC
0,Se planeo dentro del sprint la participación d...,Se deben incluir revisiones de UX en los proce...,Se planeo dentro del sprint la participación d...
1,Integrantes de parte de Sodimac debian constru...,Asegurar que la planeación y ejecución del Spr...,Integrantes de parte de Sodimac debian constru...
2,Se presentó el cambio de tres analistas de cal...,Se presentan demoras en el acoplamiento del eq...,Se presentó el cambio de tres analistas de cal...
3,Cuando en pruebas de aceptación se generaban d...,Se estableció la generación de ramas para cada...,Cuando en pruebas de aceptación se generaban d...
4,"Asesoftware participó en la definición,\n cons...","La definición, modelo de implementación técnic...","Asesoftware participó en la definición, const..."


Estandarización y tokenización del texto. Nuestra estandarización será transformar las letras mayúsculas a minúsculas y la eliminación de la puntuación. El resultado se verá en la columna __CLEANED_CONTEX_LECC__

In [8]:
data['CLEANED_CONTEX_LECC'] = data['CONTEX_LECC'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s+', ' ')
data.head()

,CONTEXTO,LECCIONES APRENDIDAS,CONTEX_LECC,CLEANED_CONTEX_LECC
0,Se planeo dentro del sprint la participación d...,Se deben incluir revisiones de UX en los proce...,Se planeo dentro del sprint la participación d...,se planeo dentro del sprint la participación d...
1,Integrantes de parte de Sodimac debian constru...,Asegurar que la planeación y ejecución del Spr...,Integrantes de parte de Sodimac debian constru...,integrantes de parte de sodimac debian constru...
2,Se presentó el cambio de tres analistas de cal...,Se presentan demoras en el acoplamiento del eq...,Se presentó el cambio de tres analistas de cal...,se presentó el cambio de tres analistas de cal...
3,Cuando en pruebas de aceptación se generaban d...,Se estableció la generación de ramas para cada...,Cuando en pruebas de aceptación se generaban d...,cuando en pruebas de aceptación se generaban d...
4,"Asesoftware participó en la definición,\n cons...","La definición, modelo de implementación técnic...","Asesoftware participó en la definición, const...",asesoftware participó en la definición constru...


Ahora, necesitaremos hacer dos pruebas, una en la que entrenaremos un modelo Word2Vec (w2v) con las stopwords y otra en la que entrenaremos otro modelo sin las stopwords. Por lo que ahora tomaremos la columna __CLEANED_CONTEX_LECC__ y eliminares las stopwords. El resuldato se podrá ver en la nueva columna llamada __WITHOUT_STOPW__

In [9]:
#txt = [cleaning(doc) for doc in nlp.pipe(data.CLEANED_CONTEX_LECC, n_threads=-1)]
brief_cleaning = data['CLEANED_CONTEX_LECC']
#txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000, n_threads=-1)]
data['WITHOUT_STOPW'] = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000, n_threads=-1)]
data.head()

,CONTEXTO,LECCIONES APRENDIDAS,CONTEX_LECC,CLEANED_CONTEX_LECC,WITHOUT_STOPW
0,Se planeo dentro del sprint la participación d...,Se deben incluir revisiones de UX en los proce...,Se planeo dentro del sprint la participación d...,se planeo dentro del sprint la participación d...,planeo sprint participación diseñadora ux incl...
1,Integrantes de parte de Sodimac debian constru...,Asegurar que la planeación y ejecución del Spr...,Integrantes de parte de Sodimac debian constru...,integrantes de parte de sodimac debian constru...,integrantes sodimac debian construir component...
2,Se presentó el cambio de tres analistas de cal...,Se presentan demoras en el acoplamiento del eq...,Se presentó el cambio de tres analistas de cal...,se presentó el cambio de tres analistas de cal...,presentó cambio analistas caldiad proyecto pre...
3,Cuando en pruebas de aceptación se generaban d...,Se estableció la generación de ramas para cada...,Cuando en pruebas de aceptación se generaban d...,cuando en pruebas de aceptación se generaban d...,pruebas aceptación generaban defectos necesari...
4,"Asesoftware participó en la definición,\n cons...","La definición, modelo de implementación técnic...","Asesoftware participó en la definición, const...",asesoftware participó en la definición constru...,asesoftware participó definición construcción ...


Entrenamiento del modelo.

In [10]:
cores = multiprocessing.cpu_count()


w2v_model = Word2Vec(workers=1, seed=123)
w2v_model_sw = Word2Vec(workers=1, seed=123)


sentences_1 = data.CLEANED_CONTEX_LECC.str.split()
sentences_2 = data.WITHOUT_STOPW.str.split()
sentences_2 = sentences_2.dropna()

#Entrenamiento del modelo con stopwords
w2v_model.build_vocab(sentences_1, progress_per=10000)
w2v_model.train(sentences_1, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)
w2v_model.init_sims(replace=True)

#Entrenamiento del modelo sin stopwords
w2v_model_sw.build_vocab(sentences_2, progress_per=10000)
w2v_model_sw.train(sentences_2, total_examples=w2v_model_sw.corpus_count, epochs=30, report_delay=1)
w2v_model_sw.init_sims(replace=True)

In [11]:
print("El modelo con stopwords tiene %d palabras en su vocabulario" % len(w2v_model.wv.vocab.keys()))
print("El modelo sin stopwords tiene %d palabras en su vocabulario" % len(w2v_model_sw.wv.vocab.keys()))

El modelo con stopwords tiene 839 palabras en su vocabulario
El modelo sin stopwords tiene 632 palabras en su vocabulario


### Miremos las 15 palabras más cercanas (similares) a _asesoftware_ para cada modelo

Modelo con stopwords

In [12]:
w2v_model.wv.most_similar(positive=['asesoftware'], topn=15)

[('ser', 0.8983310461044312),
 ('esto', 0.8982137441635132),
 ('insatisfacción', 0.882929801940918),
 ('incumplimiento', 0.8754848837852478),
 ('por', 0.8722720146179199),
 ('lideres', 0.870297372341156),
 ('ende', 0.8664594888687134),
 ('tanto', 0.8657562136650085),
 ('tenia', 0.8644993305206299),
 ('otros', 0.8603219389915466),
 ('mayor', 0.8592894673347473),
 ('compromiso', 0.8576959371566772),
 ('debido', 0.8553922176361084),
 ('pero', 0.852875828742981),
 ('otra', 0.8493099808692932)]

Modelo sin stopwords

In [13]:
w2v_model_sw.wv.most_similar('asesoftware', topn=15)

[('empresa', 0.9976372718811035),
 ('entidades', 0.9975683093070984),
 ('instalaciones', 0.9973236918449402),
 ('a', 0.9972591996192932),
 ('personal', 0.9970685243606567),
 ('soporte', 0.9968889951705933),
 ('realizadas', 0.9967949986457825),
 ('servicio', 0.996716320514679),
 ('razón', 0.9965468049049377),
 ('respuesta', 0.9964249730110168),
 ('clientes', 0.9962949752807617),
 ('ans', 0.9962491989135742),
 ('ocasiones', 0.9962224960327148),
 ('evaluar', 0.9961560964584351),
 ('mejora', 0.9960787296295166)]

De los resultados podemos descartar el modelo sin las stopwords ya que muestra que todas las palabras estas muy cercanas entre si (0.99 mínimo). Los resultados que tienen más congruencia son los del modelo entrenado con stopwords. Ahora, para dar mayor valor a estos resultados, ahora si filtremos las stopwords.

In [14]:
w2v_model.wv.most_similar(positive=['asesoftware'], topn=15)
stopwords = nlp.Defaults.stop_words

spanish_stopwords = ['y', 'e', 'a', 'o', 'u', 'tenia'] #Agregaremos esta lista a las stopwords de spacy

for word in spanish_stopwords:
    stopwords.add(word)

filter_stopwords(w2v_model.wv.most_similar('asesoftware', topn=20), stopwords)

[('insatisfacción', 0.882929801940918),
 ('incumplimiento', 0.8754848837852478),
 ('lideres', 0.870297372341156),
 ('ende', 0.8664594888687134),
 ('compromiso', 0.8576959371566772),
 ('respuesta', 0.8464267253875732),
 ('interno', 0.8460021018981934),
 ('tunja', 0.8445018529891968),
 ('inconformidad', 0.8428367376327515),
 ('realizadas', 0.8415094017982483)]

___